In [1]:
import numpy as np
import scanpy as sc
import pandas as pd
from tool import train
from tool.data import preprocessing
from tool.deconv_metric import CalDataMetric
import warnings
warnings.filterwarnings("ignore")
from tool.data import mapping_adj2matrix,adata_to_cluster_expression
from tool.calculate_adj import cal_adj,another_cal_adj

DataDir="./data/SimualtedSpatialData/dataset7/"
#DataDir="./data/SpatialcoGCN-SimData/dataset12/"
sc_file_path = DataDir+'scRNA.h5ad'
spatial_file_path = DataDir+'Spatial.h5ad'
RNA_data_adata = sc.read_h5ad(sc_file_path)
Spatial_data_adata = sc.read_h5ad(spatial_file_path)
RNA_data_adata.obs['celltype']=RNA_data_adata.obs['celltype_final']
celltype_counts = RNA_data_adata.obs['celltype'].value_counts()
print(celltype_counts)
# celltype_drop = celltype_counts.index[celltype_counts < 2]
# print(f'Drop celltype {list(celltype_drop)} contain less 2 sample')
# RNA_data_adata = RNA_data_adata[~RNA_data_adata.obs['celltype'].isin(celltype_drop),]

pancreatic A cell           443
type B pancreatic cell      171
pancreatic ductal cell      135
pancreatic acinar cell      112
pancreatic PP cell           75
pancreatic D cell            59
pancreatic stellate cell     23
endothelial cell             13
pancreatic epsilon cell       5
mast cell                     4
Name: celltype, dtype: int64


In [2]:
RNA_ret_adata=RNA_data_adata.copy()
sc.pp.normalize_total(RNA_ret_adata)
RNA_ret_adata=adata_to_cluster_expression(RNA_ret_adata)

adj_df=another_cal_adj(RNA_ret_adata,Spatial_data_adata,DataDir)
concat_st_sc=preprocessing(RNA_data_adata,Spatial_data_adata)
print(concat_st_sc)

Data exists, load it
load adj dataframe
Drop celltype [] contain less 2 sample
View of AnnData object with n_obs × n_vars = 1010 × 1619
    obs: 'celltype', 'cluster_density', 'cell_counts', 'n_genes', 'rna_count_based_density', 'batch'
    var: 'n_cells-0', 'n_cells-1'


In [4]:
adj_matrix=mapping_adj2matrix(adj_df,concat_st_sc)
ad_map=train.start(concat_st_sc,adj_matrix,GCN_method="coGCN")

In [ ]:
ad_map.obs=ad_map.obs.dropna(axis=1, how='all')
ad_map.var=ad_map.var.dropna(axis=1, how='all')
print(ad_map)
save_path=DataDir+'results/coGCN_result.h5ad'
print(save_path)
# ad_map.write(save_path)